In [18]:
import gc
import os
import pathlib
import time

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import StratifiedKFold

import lightgbm as lgb

In [2]:
%matplotlib inline

In [3]:
train_df = pd.read_csv('../input/train/train.csv')

In [4]:
acoustic_data_df = train_df[train_df.columns.drop('time_to_failure')].astype(np.int32)

In [5]:
time_to_failure_df = train_df[train_df.columns.drop('acoustic_data')].astype(np.float32)

In [6]:
del train_df
gc.collect()

18

In [7]:
acoustic_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 629145480 entries, 0 to 629145479
Data columns (total 1 columns):
acoustic_data    int32
dtypes: int32(1)
memory usage: 2.3 GB


In [8]:
time_to_failure_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 629145480 entries, 0 to 629145479
Data columns (total 1 columns):
time_to_failure    float32
dtypes: float32(1)
memory usage: 2.3 GB


In [9]:
train_values, holdout_test_values, train_target_values, holdout_test_target_values = train_test_split(
    acoustic_data_df.values,
    time_to_failure_df.values,
    test_size=0.25,
    random_state=0
)

In [10]:
linear_regression_model = LinearRegression(n_jobs=10)

In [11]:
start_time = time.time()
linear_regression_model.fit(train_values, train_target_values)
print("Total training model time: {}".format((time.time() - start_time) / 60))

Total training model time: 0.3467741847038269


In [12]:
prediction = linear_regression_model.predict(holdout_test_values)

In [13]:
print("mean_squared_error: ", mean_squared_error(holdout_test_target_values, prediction))

mean_squared_error:  13.489476553031523


In [14]:
print("mean_absolute_error: ", mean_absolute_error(holdout_test_target_values, prediction))

mean_absolute_error:  3.0482380705479266


In [15]:
#svm_regression_model = SVR()

In [16]:
#start_time = time.time()
#svm_regression_model.fit(train_values, train_target_values)
#print("Total training model time: {}".format((time.time() - start_time) / 60))

In [17]:
train_lgb_values, valid_lgb_values, train_lgb_target_values, valid_lgb_target_values = train_test_split(
    train_values,
    train_target_values,
    test_size=0.20,
    random_state=0
)

In [24]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'mae'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [22]:
lgb_train = lgb.Dataset(train_lgb_values, train_lgb_target_values.ravel())
lgb_eval = lgb.Dataset(valid_lgb_values, valid_lgb_target_values.ravel(), reference=lgb_train)

In [25]:
start_time = time.time()
gbm = lgb.train(
    params,
    lgb_train,
    num_boost_round=80,
    valid_sets=lgb_eval,
    early_stopping_rounds=10
)
print("Total training model time: {}".format((time.time() - start_time) / 60))

[1]	valid_0's l1: 3.04607
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's l1: 3.04432
[3]	valid_0's l1: 3.04272
[4]	valid_0's l1: 3.04125
[5]	valid_0's l1: 3.0399
[6]	valid_0's l1: 3.03865
[7]	valid_0's l1: 3.03751
[8]	valid_0's l1: 3.03646
[9]	valid_0's l1: 3.0355
[10]	valid_0's l1: 3.03461
[11]	valid_0's l1: 3.03379
[12]	valid_0's l1: 3.03304
[13]	valid_0's l1: 3.03235
[14]	valid_0's l1: 3.03171
[15]	valid_0's l1: 3.03112
[16]	valid_0's l1: 3.03057
[17]	valid_0's l1: 3.03006
[18]	valid_0's l1: 3.0296
[19]	valid_0's l1: 3.02917
[20]	valid_0's l1: 3.02876
[21]	valid_0's l1: 3.02839
[22]	valid_0's l1: 3.02805
[23]	valid_0's l1: 3.02773
[24]	valid_0's l1: 3.02744
[25]	valid_0's l1: 3.02716
[26]	valid_0's l1: 3.02691
[27]	valid_0's l1: 3.02667
[28]	valid_0's l1: 3.02645
[29]	valid_0's l1: 3.02625
[30]	valid_0's l1: 3.02606
[31]	valid_0's l1: 3.02588
[32]	valid_0's l1: 3.02571
[33]	valid_0's l1: 3.02556
[34]	valid_0's l1: 3.02541
[35]	valid_0's l1: 3.02528
[36]	

In [26]:
gbm_prediction = gbm.predict(holdout_test_values, num_iteration=gbm.best_iteration)

In [27]:
print("mean_absolute_error: ", mean_absolute_error(holdout_test_target_values, gbm_prediction))

mean_absolute_error:  3.0236440438006995
